
Para variables numéricas:

ANOVA (F-Statistic y p-valor): Para verificar si hay diferencias significativas en la media de la variable numérica entre las 7 categorías.

Gini impurity: Utilizando un árbol de decisión para ver qué tan bien la variable segmenta la variable objetivo.

Kruskal-Wallis: Alternativa no paramétrica a ANOVA cuando la normalidad no está garantizada.

Para variables categóricas:

Chi-cuadrado (χ² y p-valor): Para medir independencia entre la variable categórica y la variable objetivo.

IV (Information Value): Para medir la capacidad de la variable para discriminar entre categorías.

Entropía de Shannon: Para evaluar la incertidumbre de la variable respecto a la variable objetivo.

Cramer’s V: Para medir la fuerza de asociación entre la variable y la variable objetivo.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, chi2_contingency, entropy, kruskal
from sklearn.tree import DecisionTreeClassifier

def calculate_gini(x, y):
    """Calcula la importancia de Gini usando un árbol de decisión simple."""
    if x.nunique() > 1:  # Solo calcular si hay más de un valor único
        clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
        clf.fit(x.values.reshape(-1, 1), y)
        return clf.feature_importances_[0]
    return np.nan

def information_value(x, y):
    """Calcula el Information Value (IV) para una variable categórica."""
    df = pd.DataFrame({'x': x, 'y': y})
    grouped = df.groupby('x')['y'].value_counts(normalize=True).unstack().fillna(0)
    
    # Verificar si hay información suficiente
    if grouped.shape[1] < 2:  
        return np.nan
    
    woe = np.log((grouped + 0.0001) / (1 - grouped + 0.0001))  # WoE
    iv = (grouped - (1 - grouped)) * woe
    return iv.sum().sum()

def cramers_v(x, y):
    """Calcula el coeficiente de Cramer para medir la asociación entre variables categóricas."""
    contingency_table = pd.crosstab(x, y)
    
    if contingency_table.shape[0] < 2 or contingency_table.shape[1] < 2:
        return np.nan  # No se puede calcular si hay muy pocas categorías
    
    chi2_val, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    return np.sqrt(chi2_val / (n * min_dim))

def analyze_features(df, target_column):
    results = []
    y = df[target_column]  # La variable objetivo se mantiene categórica

    for column in df.columns:
        if column == target_column:
            continue
        
        x = df[column]
        result = {'Variable': column, 'Tipo': str(x.dtype)}

        # Si más del 95% de los valores son NaN, descartamos la variable
        if x.isna().sum() / len(x) > 0.95:
            result.update({'ANOVA_F': np.nan, 'Chi2': np.nan, 'IV': np.nan, 
                           'Gini': np.nan, 'Cramer_V': np.nan, 'Entropía': np.nan})
            results.append(result)
            continue

        if np.issubdtype(x.dtype, np.number):  # Variables numéricas
            groups = [x[y == cat].dropna() for cat in y.unique()]
            unique_values_per_group = [len(set(g)) > 1 for g in groups]

            if len(groups) > 1 and any(unique_values_per_group):  # Evitar errores en ANOVA y Kruskal
                f_stat, p_value = f_oneway(*groups)
                kw_stat, kw_p_value = kruskal(*groups)
                result.update({'ANOVA_F': f_stat, 'ANOVA_p': p_value, 'Kruskal_H': kw_stat, 'Kruskal_p': kw_p_value})
            else:
                result.update({'ANOVA_F': np.nan, 'ANOVA_p': np.nan, 'Kruskal_H': np.nan, 'Kruskal_p': np.nan})
            
            result['Gini'] = calculate_gini(x.dropna(), y[x.notna()])

        else:  # Variables categóricas
            if x.nunique() < 50:  # Limitar para evitar problemas en tablas muy grandes
                contingency_table = pd.crosstab(x, y)
                
                if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:  # Asegurar que hay suficientes datos
                    chi2_val, p, _, _ = chi2_contingency(contingency_table)
                    result.update({'Chi2': chi2_val, 'Chi2_p': p})
                    result['IV'] = information_value(x, y)
                    result['Cramer_V'] = cramers_v(x, y)
                    result['Entropía'] = entropy(contingency_table.sum(axis=1), base=2)
                else:
                    result.update({'Chi2': np.nan, 'Chi2_p': np.nan, 'IV': np.nan, 'Cramer_V': np.nan, 'Entropía': np.nan})
            else:
                result.update({'Chi2': np.nan, 'Chi2_p': np.nan, 'IV': np.nan, 'Cramer_V': np.nan, 'Entropía': np.nan})

        results.append(result)

    return pd.DataFrame(results)



In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, kruskal, chi2_contingency
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score

def calcular_métricas(df, target_col):
    resultados = []
    target = df[target_col]

    # Convertimos target a valores numéricos si es categórico
    target_encoded, clases = pd.factorize(target)

    for col in df.columns:
        if col == target_col:
            continue  # Omitimos la variable objetivo

        variable = df[col]
        tipo_variable = "numérica" if np.issubdtype(variable.dtype, np.number) else "categórica"

        # Inicializamos valores por defecto
        anova_p = kruskal_p = chi2_p = info_mutua = gini = "no aplica"
        bins = accuracy_por_clase = recall_por_clase = "no aplica"

        if tipo_variable == "numérica":
            temp_df = df[[col, target_col]].dropna()
            grupos = [temp_df[temp_df[target_col] == clase][col] for clase in temp_df[target_col].unique()]

            if all(len(grupo) > 1 for grupo in grupos):  # Asegurar que haya suficientes datos en cada grupo
                try:
                    anova_p = f_oneway(*grupos).pvalue  # ANOVA
                except:
                    anova_p = "error"

                try:
                    kruskal_p = kruskal(*grupos).pvalue  # Kruskal-Wallis
                except:
                    kruskal_p = "error"

            # Información Mutua
            info_mutua = mutual_info_classif(temp_df[[col]], temp_df[target_col], discrete_features=False)[0]

        elif tipo_variable == "categórica":
            temp_df = df[[col, target_col]].dropna()

            if temp_df[col].nunique() > 1:  # Asegurar que haya más de una categoría
                try:
                    tabla_contingencia = pd.crosstab(temp_df[col], temp_df[target_col])
                    chi2_p = chi2_contingency(tabla_contingencia)[1]  # Prueba de chi-cuadrado
                except:
                    chi2_p = "error"

                info_mutua = mutual_info_classif(temp_df[[col]], temp_df[target_col], discrete_features=True)[0]

        # Modelo de árbol de decisión para calcular Gini, Accuracy y Recall
        temp_df = df[[col, target_col]].dropna()
        X = temp_df[[col]].copy()
        y = temp_df[target_col]

        # Convertir categóricas a numéricas usando one-hot encoding
        if tipo_variable == "categórica":
            X_encoded = pd.get_dummies(X, drop_first=True)  # Convertir en dummies
        else:
            X_encoded = X  # Dejar las numéricas sin cambios

        # Verificar que la variable no tenga solo un valor único (para evitar errores en el modelo)
        if X_encoded.nunique().values[0] > 1:
            model = DecisionTreeClassifier(max_depth=1, random_state=42, class_weight='balanced')
            model.fit(X_encoded, y)

            gini = 2 * model.tree_.impurity[0]  # Cálculo del índice de Gini
            y_pred = model.predict(X_encoded)

            accuracy_por_clase = {}
            recall_por_clase = {}

            for i, clase in enumerate(clases):
                mascara = (y == i)
                if mascara.sum() > 0:  # Evitar divisiones por cero
                    accuracy_por_clase[clase] = accuracy_score(y[mascara], y_pred[mascara])
                    recall_por_clase[clase] = recall_score(y[mascara], y_pred[mascara], average='macro', zero_division=0)

        resultados.append({
            "variable": col,
            "tipo": tipo_variable,
            "bins/grupos": bins,
            "anova_p": anova_p,
            "kruskal_p": kruskal_p,
            "chi2_p": chi2_p,
            "info_mutua": info_mutua,
            "gini": gini,
            "accuracy_por_clase": accuracy_por_clase,
            "recall_por_clase": recall_por_clase
        })

    return pd.DataFrame(resultados)

# Cargar el dataset (reemplaza con tu archivo real)
df = pd.read_csv("tu_archivo.csv")  

# Reemplaza 'target' con el nombre real de tu variable objetivo
df_resultados = calcular_métricas(df, target_col='canal_final')

# Guardar los resultados en un archivo CSV
df_resultados.to_csv("resultados_metricas.csv", index=False)

print(df_resultados.head())


3

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def calcular_eta_squared(x, y):
    """
    Calcula el coeficiente eta squared para medir la asociación
    entre una variable numérica (x) y una categórica (y).
    """
    overall_mean = np.mean(x)
    grupos = [x[y == cat] for cat in np.unique(y)]
    ss_between = sum(len(g) * (np.mean(g) - overall_mean) ** 2 for g in grupos)
    ss_total = sum((xi - overall_mean) ** 2 for xi in x)
    return ss_between / ss_total if ss_total != 0 else np.nan

def evaluar_variables(df, target_col, n_bins=10, incluir_bins=True):
    """
    Evalúa cada variable predictora en relación con la variable objetivo,
    retornando un DataFrame con una columna por cada métrica.
    
    Parámetros:
      - df: DataFrame que contiene las variables predictoras y la variable target.
      - target_col: Nombre de la columna target (variable respuesta categórica).
      - n_bins: Número de bins a usar para variables numéricas.
      - incluir_bins: Si es True, se incluye la columna de bins/grupos en la salida.
      
    Retorna:
      - DataFrame con columnas: 'variable', 'tipo', 'bins/grupos', 'ANOVA_F',
        'ANOVA_p', 'Eta_Squared', 'Chi2', 'Chi2_p', "Cramers_V"
    """
    resultados = []
    
    for col in df.columns:
        if col == target_col:
            continue
        
        col_data = df[col]
        # Determinar el tipo de variable (numérica o categórica)
        if np.issubdtype(col_data.dtype, np.number):
            var_tipo = 'numérica'
        else:
            var_tipo = 'categórica'
        
        # Inicializamos las columnas de resultados
        bins_grupos = None  # Aquí se guardarán los bins o grupos (si se desea incluir)
        ANOVA_F = "no aplica"
        ANOVA_p = "no aplica"
        Eta_Squared = "no aplica"
        Chi2 = "no aplica"
        Chi2_p = "no aplica"
        Cramers_V = "no aplica"
        
        if var_tipo == 'numérica':
            # Binning usando pd.qcut para dividir en cuantiles
            try:
                _, bins = pd.qcut(col_data, q=n_bins, duplicates='drop', retbins=True)
                bins_grupos = bins.tolist() if incluir_bins else None
            except Exception as e:
                bins_grupos = "no aplica"
            
            # Agrupar la variable según las categorías del target
            grupos = []
            for cat in df[target_col].unique():
                grupos.append(col_data[df[target_col] == cat].dropna())
            
            # Calcular ANOVA F y p-valor si hay más de un grupo
            if len(grupos) > 1:
                try:
                    f_val, p_val = stats.f_oneway(*grupos)
                    ANOVA_F = f_val if not np.isnan(f_val) else "no aplica"
                    ANOVA_p = p_val if not np.isnan(p_val) else "no aplica"
                except Exception as e:
                    ANOVA_F, ANOVA_p = "no aplica", "no aplica"
            
            # Calcular el eta squared
            try:
                mask = col_data.notnull() & df[target_col].notnull()
                eta2 = calcular_eta_squared(col_data[mask], df[target_col][mask])
                Eta_Squared = eta2 if not np.isnan(eta2) else "no aplica"
            except Exception as e:
                Eta_Squared = "no aplica"
                
        else:  # Variable categórica
            # Los grupos son las categorías únicas
            try:
                bins_grupos = sorted(col_data.dropna().unique().tolist()) if incluir_bins else None
            except Exception as e:
                bins_grupos = "no aplica"
            
            # Crear la tabla de contingencia entre la variable y el target
            try:
                tabla = pd.crosstab(col_data, df[target_col])
                if not tabla.empty:
                    chi2, p, dof, expected = stats.chi2_contingency(tabla)
                    Chi2 = chi2 if not np.isnan(chi2) else "no aplica"
                    Chi2_p = p if not np.isnan(p) else "no aplica"
                    n = tabla.sum().sum()
                    min_dim = min(tabla.shape) - 1
                    if n * min_dim != 0:
                        Cramers_V = np.sqrt(chi2 / (n * min_dim))
                    else:
                        Cramers_V = "no aplica"
                else:
                    Chi2, Chi2_p, Cramers_V = "no aplica", "no aplica", "no aplica"
            except Exception as e:
                Chi2, Chi2_p, Cramers_V = "no aplica", "no aplica", "no aplica"
        
        resultados.append({
            'variable': col,
            'tipo': var_tipo,
            'bins/grupos': bins_grupos,
            'ANOVA_F': ANOVA_F,
            'ANOVA_p': ANOVA_p,
            'Eta_Squared': Eta_Squared,
            'Chi2': Chi2,
            'Chi2_p': Chi2_p,
            "Cramers_V": Cramers_V
        })
    
    df_resultado = pd.DataFrame(resultados)
    
    # Si no se desea incluir la columna de bins/grupos, la removemos
    if not incluir_bins:
        df_resultado.drop(columns=['bins/grupos'], inplace=True)
    
    return df_resultado

# Ejemplo de uso:
# Supongamos que tienes un DataFrame "df" y la variable target se llama "target".
# Para incluir los bins/grupos:
# df_resultado = evaluar_variables(df, target_col='target', n_bins=10, incluir_bins=True)
#
# Para omitir la columna de bins/grupos (tabla más limpia):
# df_resultado = evaluar_variables(df, target_col='target', n_bins=10, incluir_bins=False)
#
# print(df_resultado)
